# Observation History

_Rahul Biswas, Phil Marshall_

In this notebook we show how to obtain the observation history of the Twinkles field for the 2017 "baseline cadence" `minion_1016` OpSim output database. In order to run it, you would need to change the definition of `dbfileName` to the location of your local copy of an OpSim database. More OpSim databases can be downloaded from [here](https://www.lsst.org/scientists/simulations/opsim/opsim-v335-benchmark-surveys).

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [2]:
dbfileName = os.path.join('/Users/rbiswas/data/LSST/OpSimData/', 'minion_1016_sqlite.db')
engine = create_engine('sqlite:///'+dbfileName)

In [3]:
query = 'SELECT obsHistID, expMJD, Filter, FWHMeff, fiveSigmaDepth FROM Summary WHERE FIELDID is 1427 and PROPID is 54'
df = pd.read_sql_query(query, engine)

Let's look at what this returns: a table with 5 columns. Each visit has an `obsHistID`, and took place on a particular date. This timestamp is stored as `expMJD`, for "exposure mean Julian date." (For our purposes, we can assume that each visit will produce a single image, which will appears to us as an "exposure" taken by the camera. In practice, visit images would be constructed out of two separate 15-second camera exposures, but since we probably won't have access to those two images separately, we can use the terms "visit image" and "exposure" interchangeably). 

The filter can take values `ugrizy`, and the expected PSF full width at half maximum is stored as `FWHMeff`, in arcsec. This standard measure of image quality is what OpSim predicts would be the average PSF width over the whole focal plane. In practice we expect the PSF width, and shape, to vary across the field in each exposure, and hence the DM stack to provide PSF model parameters at every sky position in every visit. 
The very simple approximation of taking a single `FWHMeff` value and assuming it to be the actual width of a circularly symmetric PSF that does not vary across the field can be thought of as a placeholder: eventually we would use as much PSF information as we can get. 

The `fiveSigmaDepth` value is a prediction for the average photometric noise level across the field, and depends on the sky brightness as well as the readout noise. A point source with magnitude `fiveSigmaDepth` would be detected at 5-sigma, by having flux equal to 5 times the rms photometric noise level. This means that the flux uncertainty is given by 

`fiveSigmaDepth` = $-2.5 \log_{10}{5\sigma_f}$ 

If the `fiveSigmaDepth` is given in AB magnitudes, the flux uncertainty as defined here is in units of "AB maggies" or "mgy". In general, the flux of a source $f$ with AB magnitude $m$ is 

$f = 10^{-0.4 m}$ mgy.

In [4]:
df.head()

,obsHistID,expMJD,filter,FWHMeff,fiveSigmaDepth
0,183767,59823.286523,g,1.093153,24.377204
1,183811,59823.307264,g,1.231930,24.289872
2,184047,59823.418685,z,0.908511,21.923566
3,185595,59825.256044,r,0.949096,24.128617
4,185736,59825.325979,g,1.242407,24.316968


It'll be useful to have this table in `csv` format, so that we can easily read it back in later.

In [5]:
df.to_csv('twinkles_observation_history.csv', index=False)